<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Practice Project - Create a machine learning pipeline for a regression project


Estimated time needed: **90** minutes


## Scenario


You are a data engineer at a data analytics consulting company. Your company prides itself in being able to efficiently handle huge datasets. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able do ETL jobs and build ML pipelines.



## Objectives

In this 4 part assignment you will:

- Part 1 ETL
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Machine Learning Pipeline creation
  - Create a machine learning pipeline for prediction
- Part 3 Model evaluation
  - Evaluate the model using metrics
  - Print the intercept and the coefficients
- Part 4 Model Persistance
  - Cave the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

 - Modified version of car mileage dataset. Original dataset available at https://archive.ics.uci.edu/ml/datasets/auto+mpg 
 


----


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to
 connect to this cluster.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - ETL


### Task 1 - Import required libraries


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler

### Task 2 - Create a spark session


In [4]:
#Create SparkSession

spark = SparkSession \
    .builder \
    .appName("Practice Project") \
    .getOrCreate()

24/02/01 15:54:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Task 3 - Load the csv file into a dataframe


Download the data file


In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv

--2024-02-01 15:54:44--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14354 (14K) [text/csv]
Saving to: ‘mpg-raw.csv.1’

mpg-raw.csv.1       100%[===================>]  14.02K  --.-KB/s    in 0s      

2024-02-01 15:54:44 (32.3 MB/s) - ‘mpg-raw.csv.1’ saved [14354/14354]



Load the dataset into the spark dataframe


In [6]:
# Load dataset
df = spark.read.csv("mpg-raw.csv", header=True, inferSchema=True)

### Task 4 - Print top 5 rows of the dataset


In [7]:
df.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|46.6|        4|       86.0|        65|  2110|      17.9|  80|Japanese|
|44.6|        4|       91.0|        67|  1850|      13.8|  80|Japanese|
|44.3|        4|       90.0|        48|  2085|      21.7|  80|European|
|44.0|        4|       97.0|        52|  2130|      24.6|  82|European|
|43.4|        4|       90.0|        48|  2335|      23.7|  80|European|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



### Task 5 - Print the number of cars in each Origin


In [8]:
df.groupBy('Origin').count().orderBy('count').show()

[Stage 4:=====================================================> (195 + 5) / 200]

+--------+-----+
|  Origin|count|
+--------+-----+
|    null|    1|
|European|   70|
|Japanese|   88|
|American|  247|
+--------+-----+



### Task 6 - Print the total number of rows in the dataset


In [11]:
rowcount1 = df.count()
print(rowcount1)

406


### Task 7 - Drop all the duplicate rows from the dataset


In [14]:
df = df.dropDuplicates()

### Task 8 - Print the total number of rows in the dataset


In [15]:
rowcount2 = df.count()
print(rowcount2)
# 14 duplicates are removed

[Stage 8:==================================================>   (187 + 11) / 200]

392


### Task 9 - Drop all the rows that contain null values from the dataset


In [16]:
df = df.dropna()

### Task 10 - Print the total number of rows in the dataset


In [17]:
rowcount3 = df.count()
print(rowcount3)
# 7 na values are removed

[Stage 11:===================================================>  (191 + 8) / 200]

385


### Task 11 - Rename the column "Engine Disp" to "Engine_Disp"Drop


In [18]:
df = df.withColumnRenamed("Engine Disp", "Engine_Disp")
df.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine_Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|24.0|        4|      134.0|        96|  2702|      13.5|  75|Japanese|
|18.0|        6|      250.0|        88|  3139|      14.5|  71|American|
|29.0|        4|       68.0|        49|  1867|      19.5|  73|European|
|22.4|        6|      231.0|       110|  3415|      15.8|  81|American|
|20.5|        6|      231.0|       105|  3425|      16.9|  77|American|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



### Task 12 - Save the dataframe in parquet format, name the file as "mpg-cleaned.parquet"


In [19]:
df.write.parquet("mpg-cleaned.parquet")

[Stage 18:=======>                                               (28 + 8) / 200]24/02/01 16:08:49 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
24/02/01 16:08:49 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
24/02/01 16:08:49 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


#### Part 1 - Evaluation



Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [20]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("Renamed column name = ", df.columns[2])

import os

print("mpg-cleaned.parquet exists :", os.path.isdir("mpg-cleaned.parquet"))

Part 1 - Evaluation
Total rows =  406
Total rows after dropping duplicate rows =  392
Total rows after dropping duplicate rows and rows with null values =  385
Renamed column name =  Engine_Disp
mpg-cleaned.parquet exists : True


## Part - 2 Machine Learning Pipeline creation


### Task 1 - Load data from "mpg-cleaned.parquet" into a dataframe


In [21]:
df = spark.read.parquet("mpg-cleaned.parquet")
rowcount4 = df.count()
print(rowcount4)

[Stage 20:=====================>                                    (3 + 5) / 8]

385


In [22]:
df.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine_Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|32.2|        4|      108.0|        75|  2265|      15.2|  80|Japanese|
|28.0|        4|      107.0|        86|  2464|      15.5|  76|European|
|26.0|        4|      156.0|        92|  2585|      14.5|  82|American|
|25.0|        4|      104.0|        95|  2375|      17.5|  70|European|
|25.0|        4|      140.0|        75|  2542|      17.0|  74|American|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



In [23]:
df.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine_Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



### Task 2 - Define the StringIndexer pipeline stage


In [26]:
# Stage - 1 Using StringIndexer convert the string column "Origin" into "OriginIndex"
indexer = StringIndexer(inputCol="Origin", outputCol="OriginIndex")

### Task 3 - Define the VectorAssembler pipeline stage


In [27]:
# Stage 2 - assemble the input columns 'Cylinders','Engine_Disp','Horsepower','Weight','Accelerate','Year' into a single column "features"
assembler = VectorAssembler(inputCols=["Cylinders", "Engine_Disp", "Horsepower", "Weight", "Accelerate", "Year"], outputCol="features")

### Task 4 - Define the StandardScaler pipeline stage


In [28]:
# Stage 3 - scale the "features" using standard scaler and store in "scaledFeatures" column
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

### Task 5 - Define the StandardScaler pipeline stage


In [29]:
# Stage 4 - Create a LinearRegression stage to predict "MPG"
lr = LinearRegression(featuresCol="features", labelCol="MPG")

### Task 6 - Build the pipeline


In [30]:
# Build a pipeline using the above four stages
pipeline = Pipeline(stages=[indexer, assembler, scaler, lr])

### Task 7 - Split the data


In [31]:
# Split the data into training and testing sets with 70:30 split. Use 42 as seed
(training_data, testing_data) = df.randomSplit([0.7, 0.3], seed=42)

### Task 8 - Fit the pipeline


In [32]:
# Fit the pipeline using the training data
pipelineModel = pipeline.fit(training_data)

24/02/01 16:34:53 WARN util.Instrumentation: [75272ca2] regParam is zero, which might cause numerical instability and overfitting.
[Stage 29:>                                                         (0 + 8) / 8]24/02/01 16:34:55 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/02/01 16:34:55 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/02/01 16:34:55 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/02/01 16:34:55 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


#### Part 2 - Evaluation



Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [33]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  385
Pipeline Stage 1 =  StringIndexer
Pipeline Stage 2 =  VectorAssembler
Pipeline Stage 3 =  StandardScaler
Label column =  MPG


## Part 3 - Model Evaluation


### Task 1 - Predict using the model


In [35]:
# Make predictions on testing data
predictions = pipelineModel.transform(testing_data)

### Task 2 - Print the MSE


In [36]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)

[Stage 36:=====================>                                    (3 + 5) / 8]

10.899168410636353


### Task 3 - Print the MAE


In [37]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

[Stage 38:=======>                                                  (1 + 7) / 8]

2.6256840060950886


### Task 4 - Print the R-Squared(R2)


In [38]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)

0.8202762026606922


#### Part 3 - Evaluation



Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [39]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  10.9
Mean Absolute Error =  2.63
R Squared =  0.82
Intercept =  -13.9


## Part 4 - Model persistance


### Task 1 - Save the model to the path "Practice_Project"


In [42]:
pipelineModel.write().save("Practice_Project")

### Task 2 - Load the model from the path "Practice_Project"


In [43]:
loadedPipelineModel = PipelineModel.load("Practice_Project")

### Task 3 - Make predictions using the loaded model on the test data


In [44]:
predictions = loadedPipelineModel.transform(testing_data)

### Task 4 - Show the predictions


In [46]:
predictions.select("MPG", "prediction").show()

[Stage 70:>                                                         (0 + 1) / 1]

+----+------------------+
| MPG|        prediction|
+----+------------------+
|13.0|11.318809914955207|
|13.0|14.368910120039406|
|13.0|10.684980370653719|
|15.0|13.066592339902268|
|15.5|15.669787158219982|
|18.0| 19.81597780531899|
|18.0| 22.29980438598947|
|18.0| 20.05378878260488|
|18.6|20.890856761279917|
|19.0| 24.86218044038507|
|21.5|26.265038242692935|
|22.0| 23.09860601340578|
|23.0|21.278313625208813|
|24.0|22.969583765345686|
|25.1| 27.03028314401787|
|26.0|27.950969788034225|
|26.0|25.754412517142097|
|27.0| 28.40010908515608|
|29.0|28.123745554548893|
|30.0| 30.24946672093105|
+----+------------------+
only showing top 20 rows



#### Part 4 - Evaluation



Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [47]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[1].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  4
Coefficient for Cylinders is -0.6985
Coefficient for Engine_Disp is 0.01
Coefficient for Horsepower is -0.0016
Coefficient for Weight is -0.0063
Coefficient for Accelerate is 0.0374
Coefficient for Year is 0.7571


### Task 5 - Stop Spark Session


In [48]:
spark.stop()

Congratulations!! you have successfully finished the practice project.


## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-26|0.1|Ramesh Sannareddy|Initial Version Created|


Copyright © 2023 IBM Corporation. All rights reserved.
